
# Read from here:

### https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

What we really want is a notion of similarity between words.

It is a technique to combat the sparsity of linguistic data, by connecting the dots between what we have seen and what we haven’t. This should be captured by the deep network themselves and not be designed by the programmer 

SO WHY NOT LET THE WORD EMBEDDINGS BE THE PARAMETERS IN OUR MODEL WHICH ARE THEN UPDATED DURING TRAINING 

Word embeddings will probably not be interpretable. 

# Word Embeddings : Encoding Lexical Semantics


In [23]:
"""
word embedding in python

we need to define an index for each word 
when using embeddings. These will be the keys
into a lookup table.

Thus the embeddings will be stored as |V| X D matrix
where D is the dimensionality of the embeddings 

word i --> ith row of the matrix 

the mapping from words to indices is a dictionary
named to word_to_idx

"""

# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

word_to_ix = {"hello": 0, "world": 1}

print(word_to_ix)

embeds = nn.Embedding(3, 5)  # 2 words in vocab, 5 dimensional embeddings

# lookup_tensor returns the index 
# and converts it into long tensor
lookup_tensor = torch.tensor([word_to_ix["hello"],word_to_ix["world"]], dtype=torch.long)

# returns the row of the embedding vector
hello_embed = embeds(lookup_tensor)

print(hello_embed)
print(hello_embed.shape)

{'hello': 0, 'world': 1}
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
        [-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]])
torch.Size([2, 5])


In [2]:
"""
let us have a short description of the embedding layer

the input are (num_embeddings, embedding_dim, 
               padding_idx=None, max_norm=None, 
               norm_type=2, scale_grad_by_freq=False, 
               sparse=False, _weight=None)

read it here 
https://pytorch.org/docs/stable/nn.html#torch.nn.Embedding

num_embeddings = size of dictionary 
embedding_dim = dim of embedding

ALSO YOU CAN LOAD AN EMBEDDING
https://pytorch.org/docs/stable/nn.html#torch.nn.Embedding.from_pretrained

"""

print(embeds)



Embedding(2, 5)


# N-Gram Language Modeling


"""

Here given a sequence of words w we want to compute 
P( w_i | w_(i-1), ....., w_(i-n+1) )
where w_i is the ith word of the sequence 

we compute the loss function on some training examples 
and then update the parameters with backpropagation

"""




In [3]:

CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# We will use Shakespeare Sonnet 2

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# we should tokenize the input, but we will ignore that for now
# build a list of tuples. 
# Each tuple is ([ word_i-2, word_i-1 ], target word)
# THIS IS WHAT IS MEANT BY CONTEXT 
# WE WILL CONSIDER ONLY TWO PREVIOUS WORDS HENCE CONTEXT SIZE = 2

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]

# print the first 3, just so you can see what they look like
print(trigrams[:3])

# get the unique words 
vocab = set(test_sentence)

word_to_ix = {word: i for i, word in enumerate(vocab)}


[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [4]:
# total no of words 
print(len(test_sentence))

# total no of words in voc
print(len(set(test_sentence)))

type(trigrams)
print(len(trigrams))

for i in range(len(trigrams)):
    print(trigrams[i])
    if i+1==3:
        break

print('the length of vocab')
print(len(vocab))

"""
for i, word in enumerate(vocab):
    print(i, word)
"""

print(word_to_ix)

115
97
113
(['When', 'forty'], 'winters')
(['forty', 'winters'], 'shall')
(['winters', 'shall'], 'besiege')
the length of vocab
97
{'answer': 0, 'Were': 1, 'asked,': 2, 'sum': 48, 'praise.': 49, 'shame,': 4, 'mine': 3, 'now,': 5, 'thriftless': 51, 'a': 52, 'so': 53, 'Thy': 6, 'say,': 8, 'all-eating': 9, 'own': 10, 'and': 55, 'much': 11, 'to': 12, 'worth': 13, 'thy': 14, 'gazed': 56, "excuse,'": 15, "totter'd": 57, 'brow,': 16, 'count,': 58, 'use,': 17, 'Proving': 59, "deserv'd": 18, 'lies,': 61, 'child': 91, 'art': 19, 'How': 20, 'And': 62, 'dig': 63, 'more': 87, 'shall': 64, 'in': 65, 'proud': 22, 'my': 23, 'weed': 24, 'all': 66, 'it': 68, 'lusty': 69, 'blood': 26, 'old': 70, 'beauty': 71, 'Where': 27, 'forty': 28, 'succession': 29, 'cold.': 72, 'his': 88, 'treasure': 30, 'made': 31, 'see': 32, 'besiege': 25, 'make': 73, 'praise': 74, 'when': 75, 'If': 33, 'To': 76, 'an': 34, 'on': 35, 'trenches': 60, 'the': 77, "feel'st": 36, 'winters': 78, 'small': 95, 'livery': 79, 'within': 80, 'd

In [5]:

"""

a little explanation
embeddings layer converts the inputs into the vector
which is then passed through --> linear1 --> relu 
--> linear2 --> log_softmax

observe that the linear1 layer is like (2*embedding_dim) -->(128)
which basically means it will take the word embeddings of two words 
as input together and then convert to a 128 dim vector

"""

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        dummy_out = self.embeddings(inputs)
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs, dummy_out


In [6]:

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [7]:

print(model)


NGramLanguageModeler(
  (embeddings): Embedding(97, 10)
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)


In [8]:

for epoch in range(10):
    
    total_loss = 0
    
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model 
        # (i.e, turn the words into integer indices and wrap 
        # them in tensors)
        print(context, target)
        
        context_idxs = torch.tensor([word_to_ix[w] for w in context], 
                                    dtype=torch.long)
        
        # as context size is "2"
        print(context_idxs.shape)
        
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        
        """
        let us study this part a bit more 
        dummy_output is a torch of tensor size 
        context X embedding_dim
        
        which is converted into a torch tensor of shape
        1 X (2 X embedding_dim)
        which goes to the linear1 layer
        """
        _, dummy_output = model(context_idxs)
        print(dummy_output.shape)
        dummy_output = dummy_output.view((1, -1))
        print(dummy_output.shape)
        
        
        log_probs, _ = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor(
            [word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
        
    losses.append(total_loss)
    
print(losses)  # The loss decreased every iteration over the training data!


['When', 'forty'] winters
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['forty', 'winters'] shall
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['winters', 'shall'] besiege
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['shall', 'besiege'] thy
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['besiege', 'thy'] brow,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['thy', 'brow,'] And
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['brow,', 'And'] dig
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['And', 'dig'] deep
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['dig', 'deep'] trenches
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['deep', 'trenches'] in
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['trenches', 'in'] thy
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['in', 'thy'] beauty's
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['thy', "beauty's"] field,
torch.Size([

["totter'd", 'weed'] of
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['weed', 'of'] small
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['of', 'small'] worth
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['small', 'worth'] held:
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['worth', 'held:'] Then
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['held:', 'Then'] being
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['Then', 'being'] asked,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['being', 'asked,'] where
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['asked,', 'where'] all
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['where', 'all'] thy
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['all', 'thy'] beauty
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['thy', 'beauty'] lies,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['beauty', 'lies,'] Where
torch.Size([2])
torc

torch.Size([2, 10])
torch.Size([1, 20])
['say,', 'within'] thine
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['within', 'thine'] own
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['thine', 'own'] deep
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['own', 'deep'] sunken
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['deep', 'sunken'] eyes,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['sunken', 'eyes,'] Were
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['eyes,', 'Were'] an
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['Were', 'an'] all-eating
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['an', 'all-eating'] shame,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['all-eating', 'shame,'] and
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['shame,', 'and'] thriftless
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['and', 'thriftless'] praise.
torch.Size([2])
torch.Size([2, 10])
t

['his', 'beauty'] by
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['beauty', 'by'] succession
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['by', 'succession'] thine!
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['succession', 'thine!'] This
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['thine!', 'This'] were
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['This', 'were'] to
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['were', 'to'] be
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['to', 'be'] new
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['be', 'new'] made
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['new', 'made'] when
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['made', 'when'] thou
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['when', 'thou'] art
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['thou', 'art'] old,
torch.Size([2])
torch.Size([2, 10])
to

['thy', "beauty's"] field,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
["beauty's", 'field,'] Thy
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['field,', 'Thy'] youth's
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['Thy', "youth's"] proud
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
["youth's", 'proud'] livery
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['proud', 'livery'] so
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['livery', 'so'] gazed
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['so', 'gazed'] on
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['gazed', 'on'] now,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['on', 'now,'] Will
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['now,', 'Will'] be
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['Will', 'be'] a
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['be', 'a'] totter'd
torch.Size([2])
torch.Size([2, 

['To', 'say,'] within
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['say,', 'within'] thine
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['within', 'thine'] own
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['thine', 'own'] deep
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['own', 'deep'] sunken
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['deep', 'sunken'] eyes,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['sunken', 'eyes,'] Were
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['eyes,', 'Were'] an
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['Were', 'an'] all-eating
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['an', 'all-eating'] shame,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['all-eating', 'shame,'] and
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['shame,', 'and'] thriftless
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['and', 'thriftless'] praise.

torch.Size([2, 10])
torch.Size([1, 20])
['fair', 'child'] of
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['child', 'of'] mine
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['of', 'mine'] Shall
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['mine', 'Shall'] sum
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['Shall', 'sum'] my
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['sum', 'my'] count,
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['my', 'count,'] and
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['count,', 'and'] make
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['and', 'make'] my
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['make', 'my'] old
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['my', 'old'] excuse,'
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['old', "excuse,'"] Proving
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
["excuse,'", 'Proving'] his
t

['art', 'old,'] And
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['old,', 'And'] see
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['And', 'see'] thy
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['see', 'thy'] blood
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['thy', 'blood'] warm
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['blood', 'warm'] when
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['warm', 'when'] thou
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['when', 'thou'] feel'st
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['thou', "feel'st"] it
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
["feel'st", 'it'] cold.
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['When', 'forty'] winters
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['forty', 'winters'] shall
torch.Size([2])
torch.Size([2, 10])
torch.Size([1, 20])
['winters', 'shall'] besiege
torch.Size([2])
torch.Size(

# Computing Word Embeddins: Continuous Bag-of-Words


"""
Let us explain it in small details 

predict words given the context of a few words before and a few words 
after the target word 

not a sequential model and does not have a probabilistic interpretation

Typcially, CBOW is used to quickly train word embeddings, and these 
embeddings are used to initialize the embeddings of some more complicated 
model. Usually, this is referred to as pretraining embeddings. It 
almost always helps performance a couple of percent.

The model is like this 

given a target word w_i and N context window on each side 
i.e., w_(i-1), ..., w_(i-N) and w_(i+1), ..., w_(i+N)
refer to all context words as C

CBOW tries to minimize 
-log P(w_i|C) = -log softmax ( A * \sum_(w \in C) q_w + b)

which basically means collect all the context words and then
get their word embeddings and add them up and multiply with 
A and add bias to get the final representation 

from this compute the prob of the target word 

I BELIEVE THAT THE FINAL DIM SHOULD BE THE SIZE OF THE 
VOCABULARY
THE LOSS WILL BE NLL LOSS OR CROSS-ENTROPY LOSS

"""


In [9]:

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}


In [10]:

"""
collecting the data in terms of the context
"""

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
    
print(data[:5])

for i in range(len(data)):
    if i==5:
        break
    print(data[i])


[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
(['We', 'are', 'to', 'study'], 'about')
(['are', 'about', 'study', 'the'], 'to')
(['about', 'to', 'the', 'idea'], 'study')
(['to', 'study', 'idea', 'of'], 'the')
(['study', 'the', 'of', 'a'], 'idea')


In [11]:

class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass


In [12]:

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


print(data[0][0], data[0][1])

# data[0][0] is the input 
# data[0][1] is the target

make_context_vector(data[0][0], word_to_ix)  # example


['We', 'are', 'to', 'study'] about


tensor([41, 46, 23,  4])